<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/creating_chord_collection.git


from creating_chord_collection.collection.resources.scales import scales
from creating_chord_collection.collection.resources.transposable_figures import transposable_figures



In [2]:



# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'creating_chord_collection']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world


from creating_chord_collection.collection.chord import GuitarChord

from creating_chord_collection.collection.collection import ChordCollection







print(write_imports([]))






In [3]:
# @title Chord


# Example
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
print(chord)
print("String Frequencies:", chord.calculate_frequencies())
print("Is Open Chord:", chord.is_open())
chord.transpose(0)
print(chord.get_notes(include_strings=True))
print(chord.get_notes())

# Validate the chord against the C major scale
ionian_scale = scales["ionian"]
tonic = "C"
is_chord_in_scale = chord.validate_against_scale(tonic, ionian_scale)
print(f"Chord fits in C major scale: {is_chord_in_scale}")


('E', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
String Frequencies: {1: 329.63, 2: 246.94, 3: 196.0, 4: 164.81110255326524, 5: 123.47082531403103, 6: 82.41}
Is Open Chord: True
{1: 'E', 2: 'B', 3: 'G', 4: 'E', 5: 'B', 6: 'E'}
['E', 'B', 'G']
Chord fits in C major scale: True


In [5]:
# @title Abstract Builders
class AbstractBuilder:
    def __init__(self):
        self.image_size = None
        self.frets_endpoints = None
        self.strings_endpoints = None
        self.notes_coordenates = None
        self.name_coordenate = None
        self.font = None
        self.font_size = None

        self.root = None
        self.starting_fret = None

        self.chord_type = None
        self.finger_ascending = None

        self.scale_name = None
        self.scale = None

    def draw_image(self):
        raise NotImplementedError

    def draw_frets(self):
        raise NotImplementedError

    def draw_strings(self):
        raise NotImplementedError

    def write_starting_fret(self):
        raise NotImplementedError

    def write_name(self, name):
        raise NotImplementedError

    def draw_notes(self, notes_style):
        raise NotImplementedError

    def get_result(self):
        raise NotImplementedError

class ShortAbstractBuilder(AbstractBuilder):
    def __init__(self):
        # Image size as a tuple (width, height)
        self.image_size = (351, 351)

        # Frets endpoints as a list of tuples of tuples (each tuple represents a line start and end point)
        self.frets_endpoints = [
            ((78, 107), (273, 107)),
            ((78, 157), (273, 157)),
            ((78, 207), (273, 207)),
            ((78, 257), (273, 257)),
            ((78, 307), (273, 307))
        ]

        # Strings endpoints as a list of tuples of tuples (each tuple represents a line start and end point)
        self.strings_endpoints = [
            ((273, 107), (273, 307)),
            ((234, 107), (234, 307)),
            ((195, 107), (195, 307)),
            ((156, 107), (156, 307)),
            ((117, 107), (117, 307)),
            ((78, 107), (78, 307))
        ]

        # Notes coordinates as a dictionary for easy access
        self.notes_coordenates = {
            "strings": [273, 234, 195, 156, 117, 78],  # s1 to s6
            "frets": [82, 132, 182, 232, 282]  # f0 to f4
        }

        # Name coordinate as a tuple (x, y)
        self.name_coordenate = (175, 40)

        # Font as a string
        self.font = "arial"

        # Font size as an integer
        self.font_size = 12


class LongAbstractBuilder(AbstractBuilder):
    def __init__(self):
        self.image_size = # ...
        self.frets_endpoints = # ...
        self.strings_endpoints = # ...
        self.notes_coordenates = # ...
        self.name_coordenate = # ...
        self.font = # ...
        self.font_size = # ...



SyntaxError: invalid syntax (<ipython-input-5-c5ed8a2cca62>, line 35)

In [ ]:
# @title Concrete Builders
class ShortChordBuilder(ShortAbstractBuilder):
    def __init__(self):
        super().__init__()
        self.is_scale = False
        # Implement specific methods

class ShortScaleBuilder(ShortAbstractBuilder):
    def __init__(self):
        super().__init__()
        self.is_scale = True
        # Implement specific methods

class LongChordBuilder(LongAbstractBuilder):
    def __init__(self):
        super().__init__()
        self.is_scale = False
        # Implement specific methods

class LongScaleBuilder(LongAbstractBuilder):
    def __init__(self):
        super().__init__()
        self.is_scale = True
        # Implement specific methods


In [ ]:
# @title Director
class Director:
    def __init__(self, builder):
        self._builder = builder

    def build_chord(self, chord):
        self._builder.root = chord.root
        self._builder.chord_type = chord.chord_type
        self._builder.finger_ascending = chord.finger_ascending
        self._builder.starting_fret = chord.starting_fret
        # Call builder methods
        self._builder.draw_image()
        # ... other builder methods

    def build_scale(self, scale):
        self._builder.root = scale.root
        self._builder.scale_name = scale.name
        self._builder.starting_fret = scale.starting_fret
        self._builder.scale = scale
        # Call builder methods
        self._builder.draw_image()
        # ... other builder methods


In [ ]:
# @title Execution
# Example usage for a chord
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
builder = ShortChordBuilder()
director = Director(builder)
director.build_chord(chord)
result = builder.get_result()

# Example usage for a scale would be similar
